In [ ]:
### SQL 연습문제 9-1

# 위니브 상품정보1(weniv_product) 테이블과 위니브 상품정보2(weniv_product2)
# 테이블 간 모든 정보를 조회하시오.(단, 중복 제거하여 조회하시오.)

SELECT *
FROM `weniv.weniv_product` AS p1
UNION DISTINCT
SELECT *
FROM `weniv.weniv_product2` AS p2

In [ ]:
### SQL 연습문제 9-2

# 위니브 회원(weniv_user) 테이블에서 주문 이력(weniv_order)이 있는 사람의 id을 조회하세요.

SELECT id
FROM `weniv.weniv_user` AS t1
INTERSECT DISTINCT
SELECT user_id
FROM `weniv.weniv_order` AS t2

In [ ]:
### SQL 연습문제 9-3

# 위니브 상품정보1(weniv_product) 테이블에서 위니브 상품정보2(weniv_product2)
# 테이블에 있는 정보를 뺀 나머지의 정보를 조회하시오.

SELECT *
FROM `weniv.weniv_product`
EXCEPT DISTINCT
SELECT *
FROM `weniv.weniv_product2`

In [ ]:
### SQL 연습문제 9-4

# 상품정보(products) 테이블에서 카테고리가 ‘Clothing Sets’
# 인 정보와 'Jumpsuits & Rompers' 인 정보를 각각 조회후 합쳐서 조회하세요.

SELECT *
FROM `thelook_ecommerce.products` where category='Clothing Sets'
UNION ALL
SELECT *
FROM `thelook_ecommerce.products` where category='Jumpsuits & Rompers'

In [ ]:
### SQL 연습문제 9-5

# 회원(users) 테이블에서 국가가 'South Korea' 인 회원정보에서
# 나이가 20세~40세인 회원을 뺀 나머지 회원의 정보를 조회하시오.

SELECT *
FROM `thelook_ecommerce.users`
WHERE country = 'South Korea'
UNION ALL
SELECT *
FROM `thelook_ecommerce.users`
WHERE age >= 20
AND age <=40;       # 또는 WHERE age BETWEEN 20 AND 40;

In [ ]:
### SQL 연습문제 9-6

# 주문아이템(order_items) 테이블과 상품(products)테이블을 이용하여
# 2019 ~ 2022연도의 연도별로 매출총액이 1위인 상품의 상품명과 매출합계를 표시하세요.

# 각 연도의 매출총액 1위 상품의 상품명과 매출합계를 구한뒤
# 합쳐서 조회하세요.

# 조회 항목

# - 연도(year)
# - 상품명(product_name)
# - 매출합계금액(sum_sale_price)

# 정렬조건

# - 연도(year)

(SELECT
  2019 AS year,
  t2.name as product_name,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price
FROM `thelook_ecommerce.order_items` t1
JOIN `thelook_ecommerce.products` t2 on t1.product_id = t2.id
WHERE EXTRACT(YEAR FROM t1.created_at) = 2019
GROUP BY year, product_name
ORDER BY sum_sale_price DESC
LIMIT 1)

UNION ALL

(SELECT
  2020 AS year,
  t2.name as product_name,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price
FROM `thelook_ecommerce.order_items` t1
JOIN `thelook_ecommerce.products` t2 on t1.product_id = t2.id
WHERE EXTRACT(YEAR FROM t1.created_at) = 2020
GROUP BY year, product_name
ORDER BY sum_sale_price DESC
LIMIT 1)

UNION ALL

(SELECT
  2021 AS year,
  t2.name as product_name,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price
FROM `thelook_ecommerce.order_items` t1
JOIN `thelook_ecommerce.products` t2 on t1.product_id = t2.id
WHERE EXTRACT(YEAR FROM t1.created_at) = 2021
GROUP BY year, product_name
ORDER BY sum_sale_price DESC
LIMIT 1)

UNION ALL

(SELECT
  2021 AS year,
  t2.name as product_name,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price
FROM `thelook_ecommerce.order_items` t1
JOIN `thelook_ecommerce.products` t2 on t1.product_id = t2.id
WHERE EXTRACT(YEAR FROM t1.created_at) = 2021
GROUP BY year, product_name
ORDER BY sum_sale_price DESC
LIMIT 1)
ORDER BY year;

## 서브쿼리

In [ ]:
select id,
  first_name,
  last_name,
  (select count(order_id) from `thelook_ecommerce.orders` where user_id = a.id) as order_count
from `thelook_ecommerce.users` a
order by a.id
limit 10;

In [ ]:
WITH user_order_counts AS (
  SELECT
    user_id,
    count(order_id) AS order_count
  FROM `thelook_ecommerce.orders`
  GROUP BY user_id
), user_event_counts AS (
  SELECT
    user_id,
    count(id) AS event_count
  FROM `thelook_ecommerce.events`
  GROUP BY user_id
)
SELECT
  id,
  first_name,
  last_name,
  t2.order_count,
  t3.event_count
FROM `thelook_ecommerce.users` t1
LEFT JOIN user_order_counts t2 on t1.id = t2.user_id
LEFT JOIN user_event_counts t3 on t1.id = t3.user_id
ORDER BY t1.id

In [ ]:
### SQL 연습문제 10-1

# thelook_ecommerce 데이터 세트, events 테이블에는 유저의 웹 사이트 접속기록 데이터가 기록되어있습니다.

# 이벤트 기록에서 이벤트 타입이 purchase인 이벤트가 10번 이상 등록된 사람을 "실 고객"라고 부릅니다.

# 실고객에 해당하는 회원의 **이벤트 정보(events)**들을 유저 아이디 순으로 조회하세요.

# 실고객에 해당하는 user_id들을 먼저 구한뒤에
# 해당 user_id들을 이용해서 이벤트 정보를 조회하세요.

# - 조회 항목 : 이벤트id(id), 유저ID(user_id), 도시(city), 주(state),
# 우편번호(postal_code), 브라우저 종류(browser), 유입경로(traffic_source), 이벤트 타입(event_type)
# - 정렬 : 유저ID 오름차순

SELECT
  id,
  user_id,
  city,
  state,
  postal_code,
  browser,
  traffic_source,
  event_type
FROM `thelook_ecommerce.events`
WHERE user_id in (
  SELECT user_id
  FROM `thelook_ecommerce.events`
  where event_type = 'purchase'
  GROUP BY user_id
  HAVING count(user_id) >= 10
)
ORDER BY user_id ASC

In [ ]:
### SQL 연습문제 10-2

# thelook_ecommerce 데이터 세트 - 회원(users) 테이블과 주문(orders) 테이블에서
# 연령대별(user_count) 회원수와 주문횟수합계(order_count)를 조회하세요.

# 조회 항목

# - 연령대 (age_group)
#     - 예 : 10대, 20대, 30대, 40대 ~~ 70대
#     - 참고 함수 trunc : 소수점 버리기
#         - select trunc(78, -1) || '대'  -->  70대
#         - select trunc(32, -1) || '대'  --> 30대
# - 회원수(user_count)
# - 주문횟수합계 (order_count)

# 정렬순서

# - 연령대 오름차순

SELECT
  a.age_group,
  a.user_count,
  b.order_count
FROM
(
  SELECT
    TRUNC(age, -1) || '대' AS age_group,
    COUNT(id) AS user_count
  FROM `thelook_ecommerce.users`
  GROUP BY age_group
) a
JOIN
(
  SELECT
    TRUNC(age, -1) || '대' AS age_group,
    COUNT(id) AS order_count
  FROM `thelook_ecommerce.orders` t1
  LEFT JOIN `thelook_ecommerce.users`t2
  ON t1.user_id = t2.id
  GROUP BY age_group
) b
ON a.age_group = b.age_group
ORDER BY age_group;

In [ ]:
# 다른 방법
SELECT age_group
      ,COUNT(DISTINCT u.id) AS user_count
      ,COUNT(o.order_id) AS order_count
FROM (
    SELECT CONCAT(TRUNC(age, -1), '대') AS age_group
          ,id
    FROM `thelook_ecommerce.users`
) AS u
LEFT JOIN `thelook_ecommerce.orders` o
ON u.id = o.user_id
GROUP BY 1
ORDER BY 1;

In [ ]:
### SQL 연습문제 10-3

# thelook_ecommerce 데이터 세트, 회원(users) 테이블에서

# 1) 가장 많은 회원 나이를 구하고

# 2) 그 나이의 회원들의 정보를 조회하세요.

# 조회 항목

# - 이름(first_name, last_name)
# - 주소 정보(state, street_address, city, country)
# - 우편 번호(postal_code)

SELECT
  first_name,
  last_name,
  state,
  street_address,
  city,
  country,
  postal_code
FROM `thelook_ecommerce.users`
WHERE age = (
  SELECT MAX(age)
  FROM `thelook_ecommerce.users`
)

In [ ]:
### SQL 연습문제 10-4

# weniv 데이터 세트에서 모니터를 구입한 회원(user)의 이름(name)과
# 우편번호(postal_code)를 서브쿼리만 이용하여 조회하시오. (join을 사용하지 않습니다.)

select name, postal_code
from `weniv.weniv_user`
where id in (
  select user_id
  from `weniv.weniv_order`
  where product_id = (
    select id
    from `weniv.weniv_product`
    where name='monitor'
  )
);

# join을 사용

SELECT
  t1.name,
  t1.postal_code
FROM `weniv.weniv_user` t1
JOIN `weniv.weniv_order` t2
ON t1.id = t2.user_id
JOIN `weniv.weniv_product` t3
ON t2.product_id = t3.id
WHERE t3.name = 'monitor';

In [ ]:
### SQL 연습문제 10-5

# weniv 데이터 세트에서 소울곰(Soulgom)이라는 이름의 회원이
# 구입한 상품의 데이터를  모두 조회하세요.

# JOIN을 이용하지 않고 서브쿼리만 써서 해보세요.

SELECT *
FROM `weniv.weniv_product`
WHERE id IN (
  SELECT product_id
  FROM `weniv.weniv_order`
  WHERE User_id = (
    SELECT id
    FROM `weniv.weniv_user`
    WHERE name = 'Soulgom'
    )
)

# join을 사용

SELECT
  p.*
FROM `weniv.weniv_product` p
JOIN `weniv.weniv_order` o
  ON p.id = o.product_id
JOIN `weniv.weniv_user` u
  ON u.id = o.user_id
WHERE u.name = 'Soulgom'

In [ ]:
### SQL 연습문제 10-6

# thelook_ecommerce 데이터 세트에서 WITH절을 이용하여 상품ID, 상품명,
# 상품별 총 매출, 평균 매출, 총 주문횟수를 조회하세요.

# 1) order_items에서 상품별 총 매출, 평균매출, 주문횟수을 조회합니다. → product_sales

# - 판매금액은 sale_price 항목을 이용하여 계산하세요.

# 2) product_sales와 product를 이용하여 아래 항목을 조회하세요.

# 출력 항목

# - 상품 ID : product_id
# - 상품 명 : product_name
# - 상품 총매출 : product_total_sale_price
# - 상품 평균매출 : product_avg_sale_price
# - 상품 주문횟수 : product_order_count

WITH product_sales AS(
  SELECT
    product_id,
    SUM(sale_price) AS product_total_sale_price,
    AVG(sale_price) AS product_avg_sale_price,
    COUNT(sale_price) AS product_order_count
  FROM `thelook_ecommerce.order_items`
  GROUP BY product_id
)
SELECT
  t1.product_id,
  t2.name AS product_name,
  t1.product_total_sale_price,
  t1.product_avg_sale_price,
  t1.product_order_count
FROM product_sales t1
JOIN `weniv.weniv_product` t2
ON t1.product_id = t2.id

In [ ]:
### SQL 연습문제 11-1

# thelook_ecommerce 데이터 세트, 회원(users) 테이블에서 연령대 별 성별의 소계 및 합계를 조회하시오.

SELECT
  CONCAT(TRUNC(age, -1),'대') AS age_group, # ===  select trunc(age, -1) || '대' as age_group
  gender,
  COUNT(id) AS count_user
FROM `thelook_ecommerce.users`
GROUP BY ROLLUP(age_group, gender)
ORDER BY age_group, gender;

In [ ]:
select
  id,
  first_name,
  last_name,
  country,
  age,
  RANK() OVER ( ORDER BY age ) AS rank_number_in_all,
from thelook_ecommerce.users
where id between 1 and 20
order by age

In [ ]:
SELECT
  id,
  first_name,
  last_name,
  country,
  DENSE_RANK() OVER ( PARTITION BY country ORDER BY age ) AS dense_rank_number_in_country,
FROM `thelook_ecommerce.users`
WHERE id BETWEEN 1 AND 20
ORDER BY country, age

In [ ]:
### SQL 연습문제 11-2

# thelook_ecommerce 데이터 세트, 상품정보(products) 테이블에서 각 브랜드 내에서
# 상품 가격별 순위(brand_rank)를 조회하시오.

# - 조회 항목 : id, 브랜드(brand), 상품가격(cost), 각 브랜드 내 상품 가격이 높은 순위(brand_rank)
# - 정렬 조건 : 브랜드 오름차순, 상품가격(cost) 내림차순
# - RANK 함수를 이용해서

SELECT
  id,
  brand,
  cost,
  RANK() OVER ( PARTITION BY brand ORDER BY cost DESC ) AS brand_rank
FROM `thelook_ecommerce.products`
ORDER BY brand, cost DESC